# **Setup**

In [2]:
import os
import json
import requests

from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice

## **Load Config File**

In [3]:
# laoding the configuration file - standard way - use .env file and load_dotenv from python-dotenv module
config_file_path = "config.json"

# Read JSON data into a dictionary
with open(config_file_path, 'r') as file:
    data = json.load(file)

subscription_id = data["subscription_id"]
resource_group = data["resource_group"]  
workspace_name = data["workspace_name"]
region = data["region"]

In [4]:
print(resource_group)
print(workspace_name)
print(region)

Resource_Group_01
AzureML_workspace01
UKsouth


# **Create Azure ML Workspace**

In [5]:
# Create a workspace
ws = Workspace.create(name=workspace_name,
                      subscription_id=subscription_id,
                      resource_group=resource_group,
                      location=region)

print(f'Workspace {workspace_name} created')

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name azuremlwstoragecdeed2146.
Deploying KeyVault with name azuremlwkeyvaultdbb8bd9d.
Deploying AppInsights with name azuremlwinsights4df8f14b.
Deployed AppInsights with name azuremlwinsights4df8f14b. Took 4.33 seconds.
Deploying Workspace with name AzureML_workspace01.
Deployed Workspace with name AzureML_workspace01. Took 37.39 seconds.
Workspace AzureML_workspace01 created


# **Import ML Model**

In [6]:
# Specify the path to your  model file
model_path = 'diabetes_model.pkl'
model_name='diabetes_prediction_model'

# Register the model in Azure Machine Learning
registered_model = Model.register(model_path=model_path, model_name=model_name, workspace=ws)

Registering model diabetes_prediction_model


# **Create Environment**

In [7]:
# Create a Conda environment for your scikit-learn model
conda_env = Environment('my-conda-env')
conda_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

# **Inference Config**

In [8]:
# Create an InferenceConfig
inference_config = InferenceConfig(entry_script='score.py', environment=conda_env)

# **Create ACI**

ACI - Azure Container Instance

In [9]:
# Specify deployment configuration for ACI
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

# **Deploy ML Model**

In [10]:
service = Model.deploy(workspace=ws,
                       name='diabetes-prediction-service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config)

service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

C:\Users\aksha\AppData\Local\Temp\ipykernel_33772\2120983032.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2024-05-20 13:28:27+01:00 Creating Container Registry if not exists..
2024-05-20 13:38:27+01:00 Registering the environment..
2024-05-20 13:38:32+01:00 Building image..
2024-05-20 13:50:05+01:00 Generating deployment configuration.
2024-05-20 13:50:06+01:00 Submitting deployment to compute..
2024-05-20 13:50:13+01:00 Checking the status of deployment diabetes-prediction-service..
2024-05-20 13:51:59+01:00 Checking the status of inference endpoint diabetes-prediction-service.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://890aac3a-e61e-46a5-b6f3-51741c654251.uksouth.azurecontainer.io/score


# **Test Inference Service**

In [17]:
# Define the input data as a list
# input_data = [5,166,72,19,175,25.8,0.587,51]
input_data = [5,116,74,0,0,25.6,0.201,30]

# Convert the input data to a JSON string
input_data_json = json.dumps({"data": [input_data]})

# Replace with the URL of your deployed service
# scoring_uri = "http://82a1dd63-7ec8-49e6-8668-c222ce56cb2e.centralindia.azurecontainer.io/score"  

# Define the content type
headers = {"Content-Type": "application/json"}

# Send an HTTP POST request to the endpoint
response = requests.post(scoring_uri, data=input_data_json, headers=headers)

# Check the response status code
if response.status_code == 200:
    # Parse the JSON response
    result = json.loads(response.json())
    print(result)
    # Extract the prediction (result) from the response
    prediction = result["result"][0]
    print(f"Prediction: {prediction}")
else:
    print(f"Error: {response.text}")

{'result': [0]}
Prediction: 0


# **Delete Resources**

In [18]:
service.delete()

Running
2024-05-20 14:11:07+01:00 Check and wait for operation (ed936bd7-c536-43ef-857c-5030bdb89dc9) to finish.
2024-05-20 14:11:10+01:00 Deleting service entity.
Succeeded


In [19]:
registered_model.delete()

In [20]:
# Delete the workspace
ws.delete(delete_dependent_resources=True)